Оптимизация параметров (весов) модели
===========================

Теперь, когда у нас есть модель и данные, пришло время обучить, проверить и протестировать нашу модель, оптимизировав ее параметры на наших данных.

Обучение модели - итеративный процесс; на каждой итерации (эпохе) модель делает прогноз о выходных данных, вычисляет ошибку своего прогноза (потери), вычисляет производную ошибки по всем параметрам (то что делает autograd) и оптимизирует параметры (веса) с помощью градиентного спуска (GD).

Для более подробного ознакомления с этим процессом посмотрите видео про метод "Обратного распространения ошибки" от `3Blue1Brown` https://www.youtube.com/watch?v=tIeHLnjs5U8


Код загрузки данных и построения модели
-----------------
Мы используем код из предыдущих разделов (data, build_model).

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Гиперпараметры
-----------------

Гиперпараметры - настраиваемые параметры, которые позволяют управлять процессом оптимизации модели. Различные значения гиперпараметров могут влиять на скорость обучения модели и скорости сходимости (convergence rates).

https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html о настройке гиперпараметров

Определим следующие гиперпараметры для обучения:
 - **Number of Epochs** - количество итераций тренировки. 
 - **Batch Size** - количество объектов (пачка), которые подаются в модель за один раз.
 - **Learning Rate** - на сколько сильно обновлять веса модели в каждой эпохе. Маленькие значения приводят к медленной скорости обучения, а большие значения могут привести к непредсказуемому поведению модели.

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Цикл оптимизации
-----------------

Как только мы установим наши гиперпараметры, мы сможем обучить и оптимизировать нашу модель с помощью цикла. Каждая итерация цикла оптимизации называется **эпохой**.

Каждая эпоха состоит из 2-х главных частей:
 - **The Train Loop** - прогоняем через модель тренировочную выборку и меняем веса.
 - **The Validation/Test Loop** - прогоняем через модель тестовую выборку и проверяем насколько улучшается качество модели.

Loss Function
~~~~~~~~~~~~~~~~~

При представлении некоторой тренировочной выборки, наша необученная сетка, скорее всего, не даст правильного ответа. Функция потерь измеряет степень несоответствия полученного результата целевому значению, и именно, функцию потерь мы хотим минимизировать при обучении. Чтобы рассчитать потери, мы делаем прогноз, используя данные тренировочной и тестовой выборки, а затем сравниваем прогноз и истинное значение для каждой выборки. Так мы можем контролировать рост качества модели и отслеживать появления переобучения.

Общие функции потерь включают `nn.MSELoss `(Mean Square Error) для задач регрессии и  `nn.NLLLoss` (Negative Log Likelihood) для задач классификации.

nn.CrossEntropyLoss сочетает в себе nn.LogSoftmax и nn.NLLLoss.

Мы передаем выходные `logits` нашей модели в nn.CrossEntropyLoss, который нормализует логиты и вычисляет ошибку предсказания.


In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Оптимизатор
~~~~~~~~~~~~~~~~~

Оптимизация — это процесс настройки параметров (весов) модели для уменьшения ошибки модели на каждом этапе обучения. **Алгоритмы оптимизации** определяют, как выполняется этот процесс (в этом примере мы используем Stochastic Gradient Descent, SGD). Вся логика оптимизации заключена в объект ``optimizer``.

Здесь мы используем оптимизатор SGD. Кроме того, в PyTorch доступно множество «различных оптимизаторов», таких как ADAM и RMSProp, которые лучше работают с различными типами моделей и данных.

Мы инициализируем оптимизатор, регистрируя параметры модели, которые необходимо обучить, и передавая гиперпараметр скорости обучения.

In [7]:
for param in model.parameters():
    dt = param.data
    print(dt.size())
    print("="*100)


torch.Size([512, 784])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([10, 512])
torch.Size([10])


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Внутри цикла обучения, оптимизация происходит в 3 этапа:

 * Вызов ``optimizer.zero_grad()`` для сброса градиентов параметров (весов) модели. Градиенты по-умолчанию складываются, чтобы предотвратить двойной подсчет, мы явно обнуляем их на каждой итерации (эпохе?).

 * Обратное распространение ошибки предсказания делается с помощью вызова ``loss.backward()``. PyTorch вносит градиенты потерь относительно каждый параметр.

 * Когда у нас есть градиенты, мы вызываем ``optimizer.step()``, чтобы скорректировать параметры с помощью градиентов, собранных при `loss.backward()`.

Понлая реализация
-----------------------

Мы определяем ``train_loop``, который зацикливается на нашем коде оптимизации, и ``test_loop``, который оценивает качество модели по нашим тестовым данным.

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Мы инициализируем функцию потерь и оптимизатор и передаем их в ``train_loop`` и ``test_loop``. Не стесняйтесь увеличивать количество эпох, чтобы отслеживать улучшение качество модели.

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299706  [    0/60000]
loss: 2.290061  [ 6400/60000]
loss: 2.262267  [12800/60000]
loss: 2.259353  [19200/60000]
loss: 2.235340  [25600/60000]
loss: 2.208681  [32000/60000]
loss: 2.217761  [38400/60000]
loss: 2.173945  [44800/60000]
loss: 2.171967  [51200/60000]
loss: 2.141361  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 2.136219 

Epoch 2
-------------------------------
loss: 2.145545  [    0/60000]
loss: 2.146621  [ 6400/60000]
loss: 2.071846  [12800/60000]
loss: 2.094738  [19200/60000]
loss: 2.042636  [25600/60000]
loss: 1.979479  [32000/60000]
loss: 2.008686  [38400/60000]
loss: 1.919028  [44800/60000]
loss: 1.925777  [51200/60000]
loss: 1.863717  [57600/60000]
Test Error: 
 Accuracy: 54.5%, Avg loss: 1.856307 

Epoch 3
-------------------------------
loss: 1.887060  [    0/60000]
loss: 1.874438  [ 6400/60000]
loss: 1.733020  [12800/60000]
loss: 1.786821  [19200/60000]
loss: 1.683320  [25600/60000]
loss: 1.631230  [32000/600

Для дальнейшего изучения
-----------------------
- `Loss Functions <https://pytorch.org/docs/stable/nn.html#loss-functions>`_
- `torch.optim <https://pytorch.org/docs/stable/optim.html>`_
- `Warmstart Training a Model <https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html>`_




---